### Face detection with blurring on FPGA

### Step 1: Import stuff

In [2]:
import cv2
import time
import threading
from queue import Queue

from pynq import Overlay
from pynq import MMIO
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *

from pynq.lib import AxiGPIO

## Step 2: Define debug function

In [3]:
# print message with timed elapsed since starttime
def dtprint(msg):
    print(f"[{time.time()-dtprint.t_started:.4f}] {msg}")

dtprint.t_started = time.time()
dtprint("Started")

[0.0003] Started


### Step 3: Download overlay

In [4]:
BITFILE = "blokjes3.bit"
ol = Overlay(r"/home/xilinx/jupyter_notebooks/" + BITFILE)
ol.download()
dtprint("Overlay downloaded")

/usr/local/lib/python3.6/dist-packages/pynq/pl_server/device.py:594: UserWarning: Users will not get PARAMETERS / REGISTERS information through TCL files. HWH file is recommended.
  warnings.warn(message, UserWarning)


[10.9165] Overlay downloaded


### Step 4: Define FPGA class

In [5]:
class FPGA_Connection():
    def __init__(self):
        self.base_address = MMIO(0x43C80000,0x10000)
        self.port_x = 0x10
        self.port_y = 0x18
        self.port_w = 0x20
        self.port_h = 0x28
        self.port_mode = 0x30
        self.writeEmpty() # initialise ports to -1
        
        self.mode = None
        self.switches = getSwitches()
        self.setMode(self.switches.read())
        
    def write(self, port_address, value):
        self.base_address.write(port_address, int(value))
        
    def getSwitches(self):
        switches_ip = ol.ip_dict['switches_gpio']
        return AxiGPIO(switches_ip).channel1

    def writeSquare(self, x, y, w, h):
        #port x
        self.write(self.port_x, x)
        #port y
        self.write(self.port_y, y)
        #port w
        self.write(self.port_w, w)
        #port h
        self.write(self.port_h, h)
        
    def writeEmpty(self):
        empty_face = (-1, -1, -1, -1) # -1 since it get converted to 65535 (uint16_t)
        self.writeSquare(*empty_face)

    def setMode(self, mode):
        self.mode = mode
        dtprint(f"Mode = {mode}")
        self.write(self.port_mode, self.mode)

### Step 5: Define scan function and Scanner Thread

In [6]:
def scan(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier('/home/xilinx/jupyter_notebooks/base/video/data/'
    'haarcascade_frontalface_default.xml')    
    return face_cascade.detectMultiScale(gray, 1.3, 5)

class FullScanner(threading.Thread):

    def __init__(self, frameQ, facesQ):
        threading.Thread.__init__(self)
        self.frameQ = frameQ
        self.facesQ = facesQ
        self.running = False

    def run(self):
        while True:
            while not frameQ.empty() and self.running:
                self.facesQ.put(scan(self.frameQ.get()))

### Step 6: setup HDMI

In [7]:
hdmi_in = ol.video.hdmi_in
hdmi_out = ol.video.hdmi_out
dtprint("HDMI video in & out set")

hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)
dtprint("HDMI video in & out configured")

hdmi_in.start()
hdmi_out.start()
dtprint("HDMI video streams started")

[32.8916] HDMI video in & out set
[33.9360] HDMI video in & out configured
[34.0093] HDMI video streams started


### Step 6.5: Tie HDMI

In [8]:
hdmi_in.tie(hdmi_out)

### Step 7: Create FPGA and Scanner objects

In [9]:
FPGA = FPGA_Connection()
dtprint("Created fgpa object")

frameQ = Queue()
facesQ = Queue()

fullScanner = FullScanner(frameQ, facesQ)
fullScanner.daemon = True
fullScanner.running = False
fullScanner.start()
dtprint("started scanner thread")

[52.9313] Mode = 0
[52.9323] Created fgpa object
[52.9509] started scanner thread


### MAIN THREAD

In [14]:
frameQ.put(hdmi_in.readframe())  # fill the first frame for the scanner

fullScanner.running = True

t_started = time.time()
workTime = 30 # seconds

dtprint("started scanning")
while time.time() - t_started < workTime:
    
    if frameQ.empty() and not facesQ.empty():
        frameQ.put(hdmi_in.readframe())
        faces = facesQ.get()
        dtprint(f"#faces: {len(faces)}")
        if len(faces) > 0:
            FPGA.writeSquare(*faces[0])
        else:
            FPGA.writeEmpty()
            
    if switches.read() != FPGA.mode:
        FPGA.setMode(switches.read())
        
FPGA.setMode(0)
fullScanner.running = False
dtprint("Done!")

[275.8875] started scanning
[275.8899] Mode = 2
[277.6922] #faces: 2
[279.5612] #faces: 1
[281.5716] #faces: 1
[283.2351] #faces: 2
[285.0121] #faces: 0
[286.8927] #faces: 0
[288.6854] #faces: 0
[290.4380] #faces: 2
[292.3543] #faces: 1
[294.2327] #faces: 2
[296.1787] #faces: 2
[298.0388] #faces: 0
[300.1977] #faces: 1
[302.0708] #faces: 0
[303.9831] #faces: 0
[305.8481] #faces: 0
[305.8873] Mode = 0
[305.8898] Done!


### Shutdown everything

In [31]:
hdmi_out.close()
hdmi_in.close()

dtprint("HDMI Closed")

[2610.0921] HDMI Closed
